# **EXTRACCION DE MULTIPLES DATOS**

Se introduce los codigos a una lista iterable la cual proporciona mayor cantidad de columnas conforme se vaya aumentando la cantidad de codigos requeridos.\
*En adiccion se puede elegir entre interpolar los datos para obtener todos los registros diarios del año (0=NO, 1=SI)*

In [3]:
import pandas as pd
from datetime import datetime, timedelta
import requests

def API_BCRP(cod, interpolado):
    fecha_actual = datetime.now()
    dia_actual = fecha_actual.day
    mes_actual = fecha_actual.month
    año_actual = fecha_actual.year
    trimestre_actual = (fecha_actual.month - 1) // 3 + 1

    if cod.endswith("M"):
        Periodo_inicial = '2000-1'
        periodo_actual = f"{año_actual}-{mes_actual}"
        periodo_inicial = f"/{Periodo_inicial}/"
        periodo = periodo_inicial + periodo_actual
    elif cod.endswith("D"):
        Periodo_inicial = '2000-01-01'
        periodo_actual = f"{año_actual}-{mes_actual}-{dia_actual}"
        periodo_inicial = f"/{Periodo_inicial}/"
        periodo = periodo_inicial + periodo_actual
    elif cod.endswith("Q"):
        Periodo_inicial = "2000-1"
        periodo_actual = f"{año_actual}-{trimestre_actual}"
        periodo_inicial = f"/{Periodo_inicial}/"
        periodo = periodo_inicial + periodo_actual
    else:        
        periodo = "periodo_no_definido"

    formato = "/json"
    url_base = "https://estadisticas.bcrp.gob.pe/estadisticas/series/api/"
    url = url_base + cod + formato + periodo + '/'
    response = requests.get(url)
    response_json = response.json()
    keys = list(response_json.keys())
    diccionario_config = response_json[keys[0]]
    titulo_key = list(diccionario_config.keys())[1]
    diccionario_series = diccionario_config[titulo_key]
    diccionario_series = diccionario_series[0]
    series_key = list(diccionario_series.keys())
    Titulo = diccionario_series[series_key[0]]
    print('Tabla a extraer >>>> ' + Titulo)

    df = pd.DataFrame(response_json.get(keys[1]))
    df['values'] = df['values'].apply(lambda x: float(x[0]))

    df = df.rename(columns={'name': 'Fecha' ,'values': Titulo})

    dias = {
        'Ene': 'Jan',
        'Feb': 'Feb',
        'Mar': 'Mar',
        'Abr': 'Apr',
        'May': 'May',
        'Jun': 'Jun',
        'Jul': 'Jul',
        'Ago': 'Aug',
        'Set': 'Sep',
        'Oct': 'Oct',
        'Nov': 'Nov',
        'Dic': 'Dec'
    }
    meses = {
        'Ene': 'Jan',
        'Feb': 'Feb',
        'Mar': 'Mar',
        'Abr': 'Apr',
        'May': 'May',
        'Jun': 'Jun',
        'Jul': 'Jul',
        'Ago': 'Aug',
        'Sep': 'Sep',
        'Oct': 'Oct',
        'Nov': 'Nov',
        'Dic': 'Dec'
    }

    def convertir_fecha_dias(fecha_str):
        dia, nombre_mes, year = fecha_str.split('.')
        abreviatura_mes = dias[nombre_mes]
        fecha_str_converted = f"{abreviatura_mes}.{dia}.20{year}"
        fecha_datetime = datetime.strptime(fecha_str_converted, '%b.%d.%Y')
        return fecha_datetime

    def convertir_fecha_meses(fecha_str):
        nombre_mes, year = fecha_str.split('.')
        abreviatura_mes = meses[nombre_mes]
        fecha_datetime = datetime.strptime(abreviatura_mes + '.' + year, '%b.%Y')
        return fecha_datetime

    def convertir_fecha_trimestre(trimestre_año):
        trimestre = int(trimestre_año[1])
        año = 2000 + int(trimestre_año[3:])
        mes = (trimestre - 1) * 3 + 1
        return datetime(año, mes, 1)

    if cod.endswith("M"):
        df['Fecha'] = df['Fecha'].apply(convertir_fecha_meses)
    elif cod.endswith("D"):
        df['Fecha'] = df['Fecha'].apply(convertir_fecha_dias)
    elif cod.endswith("Q"):
        df['Fecha'] = df['Fecha'].apply(convertir_fecha_trimestre)
    else:        
        periodo = "periodo_no_definido"

    lista_cols = df.columns.tolist()
    Fecha = lista_cols[0]
    valor = lista_cols[1]

    df[Fecha] = df[Fecha].apply(pd.to_datetime)
    df = df.sort_values(by=Fecha)

    start_date = df[Fecha].iloc[0]
    end_date = df[Fecha].iloc[-1]

    all_dates = [start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)]

    prev_value_compra = None
    completado = []
    new_data = {Fecha: [], valor: []}

    for date in all_dates:
        row = df[df[Fecha] == date]
        if not row.empty:
            prev_value = row[valor].iloc[0]
            completado.append(0)
        else:
            if prev_value_compra is not None:
                completado.append(1)
            else:
                completado.append(0)
            new_data[Fecha].append(date)
            new_data[valor].append(prev_value)

    df_completed = pd.concat([df, pd.DataFrame(new_data)])
    df_completed = df_completed.sort_values(by=Fecha)

    if interpolado == 0:
        return df
    elif interpolado == 1:
        return df_completed
    else:
        print('Indicar la interpolacion')


def crear_dataframe_por_codigo(lista_codigos, interpolado):
    df_list = []
    
    for codigo in lista_codigos:
        df = API_BCRP(codigo, interpolado)
        df_list.append(df)

    if df_list:
        df_resultado = df_list[0]
        for df in df_list[1:]:
            df_resultado = pd.merge(df_resultado, df, on='Fecha', how='outer')
        return df_resultado
    else:
        return None


In [6]:
Lista_Codigos = ['PD04640PD', 'PD04639PD', 'PD04647PD', 'PD04648PD']
df_resultado = crear_dataframe_por_codigo(Lista_Codigos, 1)
df_resultado = df_resultado.fillna(0)
df_resultado

Tabla a extraer >>>> Tipo de cambio - TC Sistema bancario SBS (S/ por US$) - Venta
Tabla a extraer >>>> Tipo de cambio - TC Sistema bancario SBS (S/ por US$) - Compra
Tabla a extraer >>>> Tipo de cambio - TC Euro (S/ por Euro) - Compra
Tabla a extraer >>>> Tipo de cambio - TC Euro (S/ por Euro) - Venta


,Fecha,Tipo de cambio - TC Sistema bancario SBS (S/ por US$) - Venta,Tipo de cambio - TC Sistema bancario SBS (S/ por US$) - Compra,Tipo de cambio - TC Euro (S/ por Euro) - Compra,Tipo de cambio - TC Euro (S/ por Euro) - Venta
0,2000-01-03,3.516,3.510,0.000,0.000
1,2000-01-04,3.527,3.522,0.000,0.000
2,2000-01-05,3.520,3.513,0.000,0.000
3,2000-01-06,3.508,3.502,0.000,0.000
4,2000-01-07,3.513,3.511,0.000,0.000
...,...,...,...,...,...
8714,2023-11-12,3.813,3.806,3.723,4.277
8715,2023-11-13,3.813,3.808,3.788,4.425
8716,2023-11-14,3.777,3.765,4.066,4.228
8717,2023-11-15,3.775,3.769,4.092,4.132
